In [1]:
import os
from os.path import isfile, isdir
from tkinter import filedialog
import pandas as pd
import numpy as np
import pickle
from main_effect import main_effect
from contrast import contrast
from legacy_contrast import legacy_contrast
import compile_studies
from contribution import contribution
from folder_setup import folder_setup
from roi import check_rois
from read_exp_info import read_exp_info
from compute import *
from template import pad_shape

%matplotlib inline

#cwd = filedialog.askdirectory(title="Select analysis folder")
cwd = os.getcwd()
meta_df = pd.read_excel(f"{cwd}/DataRaw/Test.xlsx", engine='openpyxl', header=None)

if not isdir("Results"):
    folder_setup(cwd)

if isfile(f'{cwd}/Results/experiments.pickle'):
    with open(f'{cwd}/Results/experiments.pickle', 'rb') as f:
        exp_all, tasks = pickle.load(f)
else:
    exp_all, tasks = read_exp_info(f'{cwd}/DataRaw/Stroop_Updatejune2020_CWvsother_compareCundN.xlsx')

row_idx = 0
exp_name = meta_df.iloc[row_idx, 1]
exp_idxs, masks, mask_names = compile_studies.from_excel(meta_df, row_idx, tasks)
exp_df = exp_all.loc[exp_idxs].reset_index(drop=True)

num_peaks = exp_df.Peaks

bin_steps=0.0001
cluster_thresh=0.001
s0 = list(range(exp_df.shape[0]))
# highest possible ale value if every study had a peak at the same location.
mb = 1
for i in s0:
    mb = mb*(1-np.max(exp_df.at[i, 'Kernels']))

# define bins for histogram
bin_edges = np.arange(0.00005,1-mb+0.001,bin_steps)
bin_centers = np.arange(0,1-mb+0.001,bin_steps)
step = int(1/bin_steps)

peaks = np.array([exp_df.XYZ[i].T for i in s0], dtype=object)

/Users/lfrahm/.pyenv/versions/3.9.1/envs/pyALE/lib/python3.9/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
ma = np.array([exp_df.MA.values[i] for i in s0])
hx = compute_hx(s0, ma, bin_edges)
ale = compute_ale(ma)
hx_conv = compute_hx_conv(s0, hx, bin_centers, step)
z = compute_z(ale, hx_conv, step)
tfce = compute_tfce(z, sample_space)

In [4]:
%load_ext cython

In [8]:
%%cython
from mytime import tic, toc
import numpy as np
from scipy import ndimage
import nibabel as nb
cimport numpy as np
from libcpp cimport bool


cdef float E=0.6
cdef int H=2
cdef float dh = 0.4839


voxel_dims = [2,2,2]
cdef np.ndarray z = np.nan_to_num(np.array(nb.load("Results/MainEffect/Full/Volumes/Z/Other.nii").get_fdata()))
cdef float h = 0.2

cdef ctfce_par(np.ndarray[np.float_t, ndim=3] invol, float h, float dh, int voxel_size, float E=0.6, int H=2):
    cdef np.ndarray thresh 
    thresh = invol > h
    cdef np.ndarray labels
    labels, _ = ndimage.label(thresh)

    cdef np.ndarray sizes

    _, sizes = np.unique(labels, return_counts=True)
    sizes[0] = 0
    sizes = sizes * voxel_size

    cdef np.ndarray mask = labels > 0
    cdef np.ndarray szs = sizes[labels[mask]]
    cdef np.ndarray update_vals
    update_vals = np.multiply(np.power(h, H)*dh, np.power(szs, E))
    return update_vals


cdef float delta_t = np.max(z)/100
cdef np.ndarray heights = np.arange(0, np.max(z), delta_t)

start = tic()
for h in heights:
    ctfce_par(z, h, 0.4839, 8)
print(toc(start))

1.829862648999999


In [238]:
%%cython
from mytime import tic, toc
import numpy as np
from scipy import ndimage
import nibabel as nb
cimport numpy as np
from libcpp cimport bool


ctypedef np.float_t DTYPE_f
ctypedef np.int_t DTYPE_i


cdef np.ndarray voxel_dims = np.array([2,2,2])

def ctfce_par(np.ndarray[DTYPE_f, ndim=3] invol, float h, float dh, np.ndarray voxel_dims = np.array([2,2,2]), float E=0.6, int H=2):
    cdef np.ndarray[np.uint8_t, ndim = 3, cast=True] thresh = np.array(invol > h)
    cdef np.ndarray[DTYPE_i, ndim=3] labels
    
    labels, _ = ndimage.label(thresh)
    
    cdef np.ndarray[DTYPE_i, ndim=1] sizes
    
    _, sizes = np.unique(labels, return_counts=True)
    sizes[0] = 0
    sizes = sizes * np.prod(voxel_dims)
    
    cdef np.ndarray[np.uint8_t, ndim = 3, cast=True] mask = np.array(labels > 0)
    cdef np.ndarray[DTYPE_i, ndim=3] szs = sizes[labels[mask]]
    cdef np.ndarray[DTYPE_f, ndim=3] update_vals
    update_vals = np.multiply(np.power(h, H)*dh, np.power(szs, E))
    
    return update_vals


cdef np.ndarray z = np.array(nb.load("Results/MainEffect/Full/Volumes/Z/Other.nii").get_fdata())
z = np.nan_to_num(z)


cdef float delta_t = np.max(z)/100
cdef np.ndarray heights = np.arange(0, np.max(z), delta_t)
print("Print reached loop")
for h in heights:
    print(f"{h}")
    ctfce_par(z, h, delta_t)

Print reached loop
0.0


ValueError: Buffer dtype mismatch, expected 'DTYPE_i' but got 'int'

In [157]:
def tfce_par(invol, h, dh, voxel_dims=[2,2,2], E=0.6, H=2):
    thresh = np.array(invol > h)
    #look for suprathreshold clusters
    labels, cluster_count = ndimage.label(thresh)
    #calculate the size of the cluster; first voxel count, then multiplied with the voxel volume in mm
    _ , sizes = np.unique(labels, return_counts=True)
    sizes[0] = 0 
    sizes = sizes * reduce(operator.mul, voxel_dims)
    #mask out labeled areas to not perform tfce calculation on the whole brain
    mask = labels > 0
    szs = sizes[labels[mask]]
    update_vals = np.multiply(np.power(h, H)*dh, np.power(szs, E))
        
    return update_vals, mask

In [13]:
%load_ext line_profiler

In [16]:
%prun tfce_par(z, 0.3, 0.483)


         74 function calls (71 primitive calls) in 0.025 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.013    0.013    0.013    0.013 {method 'sort' of 'numpy.ndarray' objects}
        1    0.007    0.007    0.007    0.007 {built-in method scipy.ndimage._ni_label._label}
        1    0.004    0.004    0.025    0.025 tfce_par.py:6(tfce_par)
        1    0.001    0.001    0.014    0.014 arraysetops.py:310(_unique1d)
        1    0.000    0.000    0.000    0.000 {method 'flatten' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
        7    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.025    0.025 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 morphology.py:124(generate_binary_structure)
        1    0.000    0.000    0.007    0.007 measurements.py:44(label)
 

In [146]:
def compute_tfce(z, sample_space):
    
    if z.shape != shape:
        tmp = np.zeros(shape)
        tmp[tuple(sample_space)] = z
        z = tmp
    
    lc = np.min(sample_space, axis=1)
    uc = np.max(sample_space, axis=1)  
    z = z[lc[0]:uc[0],lc[1]:uc[1],lc[2]:uc[2]]
    
    delta_t = np.max(z)/100
    
    tmp = np.zeros(z.shape)
    # calculate tfce values using the parallelized function
    vals, masks = zip(*Parallel(n_jobs=-1)
                     (delayed(tfce_par)(invol=z, h=h, dh=delta_t) for h in np.arange(0, np.max(z), delta_t)))
    # Parallelization makes it necessary to integrate the results afterwards
    # Each repition creats it's own mask and an amount of values corresponding to that mask
    for i in range(len(vals)):
        tmp[masks[i]] += vals[i]
        
    tfce = np.zeros(shape)
    tfce[lc[0]:uc[0],lc[1]:uc[1],lc[2]:uc[2]] = tmp
    
    return tfce



def compute_null_cutoffs(s0, sample_space, num_peaks, kernels, step=10000, thresh=0.001, target_n=None,
                          hx_conv=None, bin_edges=None, bin_centers=None, tfce=None):
    if target_n:
        s0 = np.random.permutation(s0)
        s0 = s0[:target_n]
    # compute ALE values based on random peak locations sampled from a give sample_space
    # sample space could be all grey matter or only foci reported in brainmap
    null_ma, null_ale = compute_null_ale(s0, sample_space, num_peaks, kernels)
    # Peak ALE threshold
    null_max_ale = np.max(null_ale)
    if hx_conv is None:
        s0 = list(range(len(s0)))
        null_hx = compute_hx(s0, null_ma, bin_edges)
        hx_conv = compute_hx_conv(s0, null_hx, bin_centers, step)
    null_z = compute_z(null_ale, hx_conv, step)
    # Cluster level threshold
    null_max_cluster = compute_cluster(null_z, thresh, sample_space)
    if tfce:
        tfce = compute_tfce(null_z, sample_space)
        # TFCE threshold
        null_max_tfce = np.max(tfce)
        return null_ale, null_max_ale, null_max_cluster, null_max_tfce
        
    return null_max_ale, null_max_cluster

In [ ]:
%load_ext Cython

In [10]:
%%cython
import time
import numpy as np
cimport numpy as np


def compute_ma():
    cdef np.ndarray pad_shape = np.array([121,139,121])
    cdef np.ndarray data = np.zeros(pad_shape)
    cdef np.ndarray null_peaks = np.array([sample_space[:,np.random.randint(0,sample_space.shape[1], num_peaks[i])].T for i in s0], dtype=object)


121
